# Anatomical ROI analysis in FreeSurfer

This document includes text and images from the FreeSurfer tutorials. The full official FreeSurfer tutorials are available here:

https://surfer.nmr.mgh.harvard.edu/fswiki/Tutorials

## Introduction

This tutorial gives a brief introduction to anatomical ROI analysis, including understanding FreeSurfer label files, extracting ROI measures from label files, and creating individual and group statistics files for further analyses. 

Before we get started, we will import some general libraries and set up important paths:


In [1]:
#import necessary libraries

import os
import subprocess
from pathlib import Path
from IPython.display import display, HTML
import shutil
import pandas as pd

# Set up paths
TUTORIAL_DATA = "/home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558"
SUBJECTS_DIR = f"{TUTORIAL_DATA}/buckner_data/tutorial_subjs"

# Set environment variables for subprocess commands
os.environ["TUTORIAL_DATA"] = str(TUTORIAL_DATA)
os.environ["SUBJECTS_DIR"] = str(SUBJECTS_DIR)


## Set up a terminal window

Open a terminal windown and set up the SUBJECTS_DIR environment variable by using the following command:

**export SUBJECTS_DIR=/home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558/buckner_data/tutorial_subjs**

And now change directory to the COGNESTIC tutotial data folder:

**cd $SUBJECTS_DIR**

## Relationship between segmentation, parcellation and LookUp Table (LUT)

In this exercise, you will examine a segmentation, parcellation, and color lookup table to understand how they are related. Move to the terminal window which you set up aboce, and open a test subject (004) in Freeview using the following command and select 'coronal' view at the top menu bar:

**freeview -v 004/mri/orig.mgz 004/mri/aparc+aseg.mgz:colormap=lut:opacity=0.4 -f 004/surf/lh.white:annot=aparc.annot**

The above command opens the orig and aparc+aseg segmentation volume (aparc+aseg.mgz) as well as the cortical surface parcellation (aparc.annot) on the white surface in the left hemisphere. Note that the default parcellation uses the Desikan/Killiany atlas. There is also the option to use the Destrieux atlas parcellation, where the surface is parcellated into more anatomical regions than the Desikan/Killiany atlas. You should see something similar to the snapshot below:

![Example segmentation in coronal view](Freeview_seg_coronal.png)


Note: The aparc+aseg.mgz file shows the parcellated cortical ribbon at the same time as the segmented subcortical structures. The colormap=lut tells Freeview to display the aparc+aseg.mgz file with colors according to the look up table (LUT). The aparc+aseg.mgz uses the Desikan-Killiany atlas. To see the Destrieux atlas, you would load fsaverage/mri/aparc.a2009s+aseg.mgz 

Now lets display the contents of the LUT: 

In [2]:
# Get path to the LUT file
freesurfer_home = os.environ.get("FREESURFER_HOME")
if freesurfer_home:
    color_lut_path = Path(freesurfer_home) / "FreeSurferColorLUT.txt"
    if color_lut_path.exists():
        with open(color_lut_path) as f:
            content = f.read()
        
        # Display in a scrollable HTML box
        display(HTML(f"""
        <div style="border:1px solid #ccc; padding:10px; height:300px; overflow:auto; white-space:pre-wrap; font-family:monospace">
            {content}
        </div>
        """))
    else:
        print("Warning: FreeSurferColorLUT.txt not found.")
else:
    print("Warning: FREESURFER_HOME environment variable not set.")

### Exercise 1: Navigating between freeview and the LUT

1.	Choose the coronal view and click on a cortical structure in the brain. 
2.	See the structure name next to 'aparc+aseg' in the Cursor section below the main viewing window. For example, it may say ctx-lh-precentral. Notice which hemisphere is specified. 
3.	Look at the number listed immediately after the 'aparc+aseg'. For example, it may say 1024. 
4.	Find that value in the LUT, which you have opened using the command mentioned above. 
5.	Verify that it is the same structure you chose in freeview. 
6.	Do the same with a subcortical structure of your choice. 

You can close freeview once you are done. 



## Individual Stats files

During the FreeSurfer processing stream, via the recon-all script, some statistical output files are generated. They are kept in each subjects' stats/ subdirectory and are generated for the subcortical segmentation (aseg) and the cortical parcellation (aparc). These tables include information on each labeled region for the individual subject. 

You can view these output files via the terminal or a text editor. 

### aseg.stats

The statistical output from the subcortical segmentation, called aseg.stats, is a regular text file and will contain the volumes of specific structures. For example, you can obtain information such as the volume of left hippocampus and its mean intensity from this file. 


In [3]:
# Look through the aseg.stats file for subject 004
subject_id = "004"
aseg_stats_path = Path(SUBJECTS_DIR) / subject_id / "stats" / "aseg.stats"

# Display full contents in a scrollable box
if aseg_stats_path.exists():
    with open(aseg_stats_path) as f:
        content = f.read()

    display(HTML(f"""
    <div style="border:1px solid #ccc; padding:10px; height:400px; overflow:auto; white-space:pre-wrap; font-family:monospace">
        {content}
    </div>
    """))
else:
    print(f"Warning: aseg.stats not found for subject {subject_id}.")

At the head of the text file there will be information about the command that was run, the version used, the user who ran it and a time stamp. Following this there is information about the volume of the entire brain. 

The next section of this file defines the column headers, field name, and units for the rest of the table. We can expect to see the Segmentation Id, Number of Voxels, Volume, Structure Name, Intensity normMean, Intensity normStdDev, Intensity normMin, Intensity normMax, and Intensity normRange for each entry in the table. 

The "norm" stats are extracted for each segmented structure from $SUBJECTS_DIR/004/mri/norm.mgz.

The remainder of the table shows this information for all the structures that are labeled in the aseg. If you open the aseg.stats file with a text editor, you may find the various headings don't line up perfectly. This is because the text file is formatted for spreadsheet programs. 

### aparc.stats

The statistical output from the cortical parcellation, called lh.aparc.stats and rh.aparc.stats, is a regular text file and will contain the thickness of specific structures. For example, you can obtain information such as, how big is left superior temporal gyrus and its average thickness from this file. 



In [4]:
# Look through the lh.aparc.stats file for subject 004
subject_id = "004"
lh_aparc_stats_path = Path(SUBJECTS_DIR) / subject_id / "stats" / "lh.aparc.stats"

# Display full contents in a scrollable box
if lh_aparc_stats_path.exists():
    with open(lh_aparc_stats_path) as f:
        content = f.read()

    display(HTML(f"""
    <div style="border:1px solid #ccc; padding:10px; height:400px; overflow:auto; white-space:pre-wrap; font-family:monospace">
        {content}
    </div>
    """))
else:
    print(f"Warning: lh.aparc.stats not found for subject {subject_id}.")

This file takes the same format as the aseg.stats. The measures at the top show the number of vertices in the cortex (NumVert) and the surface area of the cortex (SurfArea). This part of the file also tells us that the lh.aparc.annot is being used as the annotation file (AnnotationFile ../label/lh.aparc.annot).

The next section of this file defines the column headers, field name, and units for the rest of the table. We can expect to see the Structure Name, Number of Vertices, Surface Area, Gray Matter Volume, Average Thickness, Thickness StDev, Integrated Rectified Mean Curvature, Integrated Rectified Gaussian Curvature, Folding Index and Intrinsic Curvature Index for each entry in the table.

The remainder of the table shows this information for all the structures that are labeled in the aparc. 

## Group stats files

This section will run you through using the stats directory of the subjects to perform group stats of certain structures that may be of interest to your study. The following commands will help you combine the data of the subjects you are analyzing into one table that will be easily read into a spreadsheet program. We have considered 6 subjects as examples (004, 021, 040, 067, 080, 092) in the following sections. 

### Table of segmentation volumes

This section explains how to create a table of segmentation volumes using the 6 subjects mentioned above. 



In [5]:
# Generate aseg volume table
sub_ids = ["004", "021", "040", "067", "080", "092"]
seg_nos = ["11", "17", "18"]
aseg_table = "aseg.vol.table"

# Build the command
aseg_cmd = [
    "asegstats2table",
    "--subjects", *sub_ids,
    "--segno", *seg_nos,
    "--tablefile", aseg_table
]

# Run the command
subprocess.run(aseg_cmd, check=True)
print(f"\nCreated aseg volume table (segno 11, 17, 18): {aseg_table}")

SUBJECTS_DIR : /home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558/buckner_data/tutorial_subjs
Parsing the .stats files
Building the table..
Writing the table to aseg.vol.table

Created aseg volume table (segno 11, 17, 18): aseg.vol.table


The input for the --segno flag correspond to the segmentation label of left caudate, left hippocampus, and left amygdala, respectively. (You can create a table with all of the labels, not just these three, by omitting the --segno part.) 

Use the link below if you would like to view the list of labels and their corresponding Look Up Table ID numbers again:

https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT

The file aseg.vol.table is your output - a text file consisting of the subjects mentioned in the command above and the values for the structures requested along with the measures in the header. The information in this text file is formatted so it can be easily imported into a spreadsheet program (often used as input for many statistical analysis programs). 

Next we can have a look at the contents of this file:


In [6]:
# Read and display aseg.vol.table
aseg_df = pd.read_csv(aseg_table, sep='\t')  # assumes tab-separated file
aseg_df.head(6)  # displays the first six rows (all subject in this case since only 6 selected)

,Measure:volume,Left-Caudate,Left-Hippocampus,Left-Amygdala,BrainSegVol,BrainSegVolNotVent,BrainSegVolNotVentSurf,lhCortexVol,rhCortexVol,CortexVol,...,SupraTentorialVol,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,MaskVol,BrainSegVol-to-eTIV,MaskVol-to-eTIV,lhSurfaceHoles,rhSurfaceHoles,SurfaceHoles,EstimatedTotalIntraCranialVol
0,4,5379.5,3917.1,1316.9,1262276.0,1187271.0,1.186952e+06,259221.573235,258713.044172,517934.617407,...,1.131347e+06,1.062304e+06,1060851.0,1740098.0,0.701763,0.967408,70.0,59.0,129.0,1.798722e+06
1,21,4342.2,4150.2,1996.0,1272062.0,1246030.0,1.245705e+06,273668.647340,272968.502464,546637.149805,...,1.129017e+06,1.107437e+06,1106044.0,1695636.0,0.763237,1.017381,39.0,28.0,67.0,1.666668e+06
2,40,2569.5,3280.7,1353.1,1006983.0,992648.0,9.928552e+05,214479.952096,213080.261300,427560.213395,...,8.805122e+05,8.694992e+05,867676.0,1379027.0,0.787259,1.078123,14.0,22.0,36.0,1.279100e+06
3,67,4096.2,3929.7,1716.6,1220261.0,1147349.0,1.147058e+06,240870.499705,237371.496658,478241.996363,...,1.074372e+06,1.007461e+06,1006223.0,1730524.0,0.679431,0.963541,46.0,53.0,99.0,1.796005e+06
4,80,4049.6,3833.2,1600.0,1367604.0,1343620.0,1.343809e+06,300158.481932,299363.718508,599522.200439,...,1.207408e+06,1.187552e+06,1185664.0,1874737.0,0.765084,1.048791,34.0,34.0,68.0,1.787521e+06
5,92,3317.2,4086.2,1658.6,1180941.0,1149674.0,1.149756e+06,231947.635639,233608.275221,465555.910860,...,1.054766e+06,1.026603e+06,1024781.0,1643820.0,0.756169,1.052556,11.0,20.0,31.0,1.561742e+06


In the table, the first cell is volume indicating that the measure is a volume in mm3 for all of the cells to the right. The subject IDs can be found below volume (seen as 4, 21, 40, 67, 80, 92). You'll notice that in the examples we've considered here for asegstats2table, each subject is a 3 digit number (004, 021 etc).

### Table of white matter parcellation volumes

The purpose of this section is to show how you can change which segmentation atlas you get stats from (and thus which structures): 


In [7]:
# Generate white matter parcellation volume table

# Define subjects and segmentation IDs for wmparc
seg_ids = ["3007", "3021", "3022", "4022"]
wmparc_table = "wmparc.vol.table"

# Build the command
wmparc_cmd = [
    "asegstats2table",
    "--subjects", *sub_ids,
    "--segno", *seg_ids,
    "--stats", "wmparc.stats",
    "--tablefile", wmparc_table
]

# Run the command
subprocess.run(wmparc_cmd, check=True)
print(f"\nCreated white matter parcellation volume table: {wmparc_table}")

SUBJECTS_DIR : /home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558/buckner_data/tutorial_subjs
Parsing the .stats files
Building the table..
Writing the table to wmparc.vol.table

Created white matter parcellation volume table: wmparc.vol.table


### Table of the mean thickness of each cortical parcellation in the Desikan/Killiany atlas

This section demonstrates how to create a table of the mean thickness of each cortical parcellation in the DDesikan/Killiany atlas:


In [8]:
# Generate left hemisphere cortical thickness table
aparc_table = "lh.aparc.thickness.table"

# Build the command
aparc_cmd = [
    "aparcstats2table",
    "--hemi", "lh",
    "--subjects", *sub_ids,
    "--parc", "aparc",
    "--meas", "thickness",
    "--tablefile", aparc_table
]

# Run the command
subprocess.run(aparc_cmd, check=True)
print(f"\nCreated left hemisphere cortical thickness table: {aparc_table}")

SUBJECTS_DIR : /home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558/buckner_data/tutorial_subjs
Parsing the .stats files
Building the table..
Writing the table to lh.aparc.thickness.table

Created left hemisphere cortical thickness table: lh.aparc.thickness.table


### Exercise 2: Inspect the contents of the lh.aparc.thickness.table file

1. Open a new cell below and write some code to display the contents of the cortical thinkness summary table you created above.  